In [1]:
# baseline cnn model for digit
import numpy as np
import import_ipynb
from loader import Loader
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization



importing Jupyter notebook from loader.ipynb


In [2]:
from matplotlib import pyplot

In [3]:
def dataset1():
# Dividing the data into train and test data
    dataset = Loader()
    dataset.pca()
    trainX, trainY = dataset.getWholeTrainSet(pca=False)
    testX, testY = dataset.getWholeTestSet(pca=False)
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
    
    trainX = trainX.reshape((trainX.shape[0], 16,15, 1))
    testX = testX.reshape((testX.shape[0], 16,15, 1))
    testY = to_categorical(testY)
    trainY = to_categorical(trainY)
    return trainX, trainY, testX, testY



In [ ]:
#Data normalization
def normalise_data(traindata, testdata):
    traindata = traindata.astype('float32')
    testdata = testdata.astype('float32')
    train_norm = traindata / 6.0
    test_norm = testdata / 6.0
    return train_norm, test_norm

In [4]:
def model_definition():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(16,15, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
#    model.add(Dropout(0.50))
    

#    model.add(Conv2D(16, (2, 2), activation='relu', kernel_initializer='he_uniform', input_shape=(16,15, 1)))
#    model.add(BatchNormalization())
#    model.add(MaxPooling2D((2, 2)))


    model.add(Flatten())
    
    # Flatten and apply drop out or apply drop out after Conv2D it is essentially the same
    model.add(Dropout(0.50))
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
  #  model.add(Dropout(0.20))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    model.add(Dropout(0.20))
    model.add(Dense(10, activation='softmax'))
    opt = SGD(lr=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
def train_model(mtrainX, mtrainY, folds=5):
    accuracies = list()
    # random shuffle k fold validation
    kfold = KFold(folds, shuffle=True, random_state=3)
    for training_indices, testing_indices in kfold.split(mtrainX):
        model = model_definition()
        trainX, trainY, testX, testY = mtrainX[training_indices], mtrainY[training_indices], mtrainX[testing_indices], mtrainY[testing_indices]
        model.fit(trainX, trainY, epochs=120, batch_size=10, validation_data=(testX, testY), verbose=0)
        _, accuraccy = model.evaluate(testX, testY, verbose=0)
        print('%.3f' % (accuraccy * 100.0))
        # stores accuracies and test data on last kth model
        accuracies.append(accuraccy)
    return accuracies, model

In [6]:
def run_model():
    trainX, trainY, testX, testY = dataset1()
 #   trainX, testX = normalise_data(trainX, testX)
    # training model
    accuracies,model = train_model(trainX, trainY)
    # Test model on kth trained model
    _, accuracy1 = model.evaluate(testX, testY, verbose=0)
    print('Test accuracy =',accuracy1*100.0)
    return accuracy1


In [7]:
run_model()

97.500
96.500
98.500
97.500
99.000
Test accuracy = 97.79999852180481


0.9779999852180481